In [ ]:
import os
import gc
import json
import torch
import pandas as pd
from tqdm import tqdm
from datasets import load_dataset, Dataset
from vllm import LLM, SamplingParams
import time
import torch.multiprocessing as mp


device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

# Set environment variable to help with memory fragmentation
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

# Track progress
PROGRESS_FILE = "last_processed.json"

/home/anamikaghosh_umass_edu/.conda/envs/gpu-env/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-04-05 15:57:28,730	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


Using device: cuda


In [2]:
def load_progress():
    if os.path.exists(PROGRESS_FILE):
        with open(PROGRESS_FILE, "r") as f:
            return json.load(f)
    return {}

def save_progress(progress_dict):
    with open(PROGRESS_FILE, "w") as f:
        json.dump(progress_dict, f)

def clean_output(text, keyword):
    index = text.rfind(keyword)
    if index != -1:
        return text[index + len(keyword):].strip()
    return text

In [3]:
class ExplanationGeneratorLama:
    def __init__(self, model_name, max_new_tokens=500):
        self.max_new_tokens = max_new_tokens
        self.sampling_params = SamplingParams(
            temperature=0.7,
            top_p=0.95,
            max_tokens=max_new_tokens
        )
        self.llm = LLM(
            model=model_name,
            dtype="half"
        )

    def build_prompts(self, entry):
        prompt_templates = [
            "Instruction: Provide a concise explanation of what the above code mean. Generate strictly less than 100 words in total. Please give the output just as text only. Do not return anything else. Answer: \n",
            "Instruction: Provide a detailed line-by-line explanation of this code snippet, describing the purpose and functionality of each statement, function, and control structure. Please give the output just as text only. Do not return anything else. Answer: \n",
            "Instruction: Summarize what this code snippet does in simple, non-technical language, focusing on its overall purpose and key operations for someone with little programming experience. Please give the output just as text only. Do not return anything else. Answer: \n",
            "Instruction: Generate an explanation of the code snippet in such a way that it can regenerate the code based on this explanation. Please give the output just as text only. Do not return anything else. Answer: \n",
            "Instruction: Explain how the code snippet  is implemented. Please provide the explanation as text only without any additional content. Answer: \n"
        ]
        prompts = []
        for template in prompt_templates:
            prompt = (
                f"Code snippet: {entry}\n"
                f"{template}"
            )
            prompts.append(prompt)
        return prompts

    def generate_explanations_batch(self, batch):
        all_prompts = []
        print(batch)
        print(type(batch))
        for entry in batch['code']:
            all_prompts.extend(self.build_prompts(entry))
        results = self.llm.generate(all_prompts, self.sampling_params)
        explanations = [res.outputs[0].text for res in results]

        # Group explanations
        grouped = [explanations[i:i+5] for i in range(0, len(explanations), 5)]
        return grouped

def process_and_save(batch, generator, model_key, output_path, progress_dict):
    explanations = generator.generate_explanations_batch(batch)
    new_data = []
    for i, entry in enumerate(batch):
        row = {
            "corpus_id": entry["corpus_id"],
            "query_id": entry["query_id"],
            "doc": entry["doc"],
            "code": entry["code"]
        }
        for j, explanation in enumerate(explanations[i]):
            row[f"explanation_{model_key}_{j+1}"] = explanation
        new_data.append(row)

    df_batch = pd.DataFrame(new_data)
    df_batch.to_csv(output_path, mode='a', index=False, header=not os.path.exists(output_path))

    # Only update progress after successful write
    last_id = batch[-1]["corpus_id"]
    progress_dict[model_key] = last_id
    save_progress(progress_dict)

In [ ]:
if __name__ == "__main__":
    mp.set_start_method("spawn", force=True)
    start = time.time()
    csv_path = "/work/pi_wenlongzhao_umass_edu/27/anamikaghosh/data_preprocessing/CodeSearchNet_Python_valid.csv"
    output_path = "/work/pi_wenlongzhao_umass_edu/27/anamikaghosh/explanations/CodeSearchNet_Python_valid_vllm_new.csv"
    
    dataset = Dataset.from_pandas(pd.read_csv(csv_path))

    models_dict = {
        "deepseek": "/datasets/ai/deepseek/hub/models--deepseek-ai--DeepSeek-R1-Distill-Qwen-1.5B/snapshots/530ca3e1ad39d440e182c2e4317aa40f012512fa",
        # "granite": "/datasets/ai/ibm-granite/hub/models--ibm-granite--granite-3.0-2b-instruct/snapshots/69e41fe735f54cec1792de2ac4f124b6cc84638f"
    }

    batch_size = 40  # effective batch is 5 * 40 = 200

    progress_dict = load_progress()

    for model_key, model_path in tqdm(models_dict.items()):
        print(f"\nProcessing model {model_key}")
        generator = ExplanationGeneratorLama(model_path)

        # Determine where to resume
        last_id = progress_dict.get(model_key)
        if last_id:
            idx = next((i for i, ex in enumerate(dataset) if ex['corpus_id'] == last_id), -1)
            if idx != -1:
                dataset = dataset.select(range(idx + 1, len(dataset)))

        def process_batch(batch):
            process_and_save(batch, generator, model_key, output_path, progress_dict)
            return batch  # Return unmodified batch for `map`'s sake

        dataset.map(
            process_batch,
            batched=True,
            batch_size=batch_size,
            num_proc=5,
            load_from_cache_file=False
        )

        del generator
        gc.collect()
        torch.cuda.empty_cache()

    end = time.time()
    print(f"All model generations written to {output_path}")
    print(f'Overall time taken = {(end-start)} seconds')
    


  0%|          | 0/1 [00:00<?, ?it/s]


Processing model deepseek
INFO 04-05 15:57:30 __init__.py:207] Automatically detected platform cuda.
WARNING 04-05 15:57:30 config.py:2448] Casting torch.bfloat16 to torch.float16.
INFO 04-05 15:57:38 config.py:549] This model supports multiple tasks: {'embed', 'reward', 'score', 'classify', 'generate'}. Defaulting to 'generate'.
WARNING 04-05 15:57:38 arg_utils.py:1187] Chunked prefill is enabled by default for models with max_model_len > 32K. Currently, chunked prefill might not work with some features or models. If you encounter any issues, please disable chunked prefill by setting --enable-chunked-prefill=False.
INFO 04-05 15:57:38 config.py:1555] Chunked prefill is enabled with max_num_batched_tokens=2048.
INFO 04-05 15:57:38 llm_engine.py:234] Initializing a V0 LLM engine (v0.7.3) with config: model='/datasets/ai/deepseek/hub/models--deepseek-ai--DeepSeek-R1-Distill-Qwen-1.5B/snapshots/530ca3e1ad39d440e182c2e4317aa40f012512fa', speculative_config=None, tokenizer='/datasets/ai/de

[W405 15:57:40.630285986 CUDAAllocatorConfig.h:28] Warning: expandable_segments not supported on this platform (function operator())

Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]
Loading safetensors checkpoint shards: 100% Completed | 1/1 [00:00<00:00,  1.31it/s]



INFO 04-05 15:57:41 model_runner.py:1115] Loading model weights took 3.3414 GB
INFO 04-05 15:57:41 worker.py:267] Memory profiling takes 0.48 seconds
INFO 04-05 15:57:41 worker.py:267] the current vLLM instance can use total_gpu_memory (44.52GiB) x gpu_memory_utilization (0.90) = 40.07GiB
INFO 04-05 15:57:41 worker.py:267] model weights take 3.34GiB; non_torch_memory takes 0.08GiB; PyTorch activation peak memory takes 1.39GiB; the rest of the memory reserved for KV Cache is 35.25GiB.
INFO 04-05 15:57:42 executor_base.py:111] # cuda blocks: 82516, # CPU blocks: 9362
INFO 04-05 15:57:42 executor_base.py:116] Maximum concurrency for 131072 tokens per request: 10.07x
INFO 04-05 15:57:44 model_runner.py:1434] Capturing cudagraphs for decoding. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI. If out-of-memory error occurs during cudagraph capture, consider decreasing `gpu_memory_

Capturing CUDA graph shapes: 100%|██████████| 35/35 [00:14<00:00,  2.43it/s]

...


<class 'datasets.formatting.formatting.LazyBatch'>


Processed prompts:   0%|          | 0/200 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]


{'Unnamed: 0': [11031, 11032, 11033, 11034, 11035, 11036, 11037, 11038, 11039, 11040, 11041, 11042, 11043, 11044, 11045, 11046, 11047, 11048, 11049, 11050, 11051, 11052, 11053, 11054, 11055, 11056, 11057, 11058, 11059, 11060, 11061, 11062, 11063, 11064, 11065, 11066, 11067, 11068, 11069, 11070], 'query_id': ['q262952', 'q262953', 'q262954', 'q262955', 'q262956', 'q262957', 'q262958', 'q262959', 'q262960', 'q262961', 'q262962', 'q262963', 'q262964', 'q262965', 'q262966', 'q262967', 'q262968', 'q262969', 'q262970', 'q262971', 'q262972', 'q262973', 'q262974', 'q262975', 'q262976', 'q262977', 'q262978', 'q262979', 'q262980', 'q262981', 'q262982', 'q262983', 'q262984', 'q262985', 'q262986', 'q262987', 'q262988', 'q262989', 'q262990', 'q262991'], 'corpus_id': ['c262851', 'c262852', 'c262853', 'c262854', 'c262855', 'c262856', 'c262857', 'c262858', 'c262859', 'c262860', 'c262861', 'c262862', 'c262863', 'c262864', 'c262865', 'c262866', 'c262867', 'c262868', 'c262869', 'c262870', 'c262871', 'c26

  0%|          | 0/1 [00:31<?, ?it/s]


RuntimeError: Cannot re-initialize CUDA in forked subprocess. To use CUDA with multiprocessing, you must use the 'spawn' start method